In [1]:
using Pkg

Pkg.activate(".")

  Activating project at `~/Documents/Uni/Master/MA/visual_analysis`


In [2]:
using GLMakie
using GeoMakie
using EmpiricalOrthogonalFunctions
using NCDatasets 
using Dates
using BenchmarkTools
using Statistics
using StatsBase

In [3]:
include("utils.jl")

┌ Warning: attempting to remove probably stale pidfile
│   path = /home/denis/.julia/compiled/v1.10/PythonCall/WdXsa_gTjUH.ji.pidfile
└ @ FileWatching.Pidfile /home/denis/.julia/juliaup/julia-1.10.3+0.x64.linux.gnu/share/julia/stdlib/v1.10/FileWatching/src/pidfile.jl:244
    CondaPkg Found dependencies: /home/denis/Documents/Uni/Master/MA/visual_analysis/CondaPkg.toml
    CondaPkg Found dependencies: /home/denis/.julia/packages/PythonCall/bb3ax/CondaPkg.toml
    CondaPkg Dependencies already up to date


calculate_eofs_of_tl_data (generic function with 1 method)

In [4]:
mon_mean_base_path = "/home/denis/workspace/data/ivt_monthly_mean"
day_mean_base_path = "/home/denis/workspace/data/ivt_daily_mean"
ps_data_monthly = "/home/denis/workspace/data/ps_data_monthly"

"/home/denis/workspace/data/ps_data_monthly"

In [5]:
# base_path = "/home/denis/workspace/data/ivt_fields_v1"
base_path = "/mnt/bigdrive/Datasets/ivt_fields_v1"
# base_path = "/home/denis/workspace/datasets/ps_data_fast"
monthly_ps_data_path = "/home/denis/workspace/data/ps_data_monthly"
mon_mean_base_path = "/home/denis/workspace/data/ivt_monthly_mean"


"/home/denis/workspace/data/ivt_monthly_mean"

In [6]:
winter_months = [12, 1, 2, 3]

4-element Vector{Int64}:
 12
  1
  2
  3

In [13]:
ps_data = filter_by_date(build_timeline_data(monthly_ps_data_path, 10, "ssp126", "ssp585"; file_range_selection = :, data_field_id="ps")) do time_element

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-16T12:00:00"), DateTime("2015-02-14T21:00:00"), DateTime("2015-03-16T09:00:00"), DateTime("2015-12-16T09:00:00"), DateTime("2016-01-16T09:00:00"), DateTime("2016-02-15T09:00:00"), DateTime("2016-03-16T09:00:00"), DateTime("2016-12-16T09:00:00"), DateTime("2017-01-16T09:00:00"), DateTime("2017-02-14T21:00:00")  …  DateTime("2098-12-16T09:00:

In [8]:

monthly_tl_data = filter_by_date(build_timeline_data(mon_mean_base_path, 10, "ssp126", "ssp585"; file_range_selection = :)) do time_element

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-16T12:00:00"), DateTime("2015-02-14T21:00:00"), DateTime("2015-03-16T09:00:00"), DateTime("2015-12-16T09:00:00"), DateTime("2016-01-16T09:00:00"), DateTime("2016-02-15T09:00:00"), DateTime("2016-03-16T09:00:00"), DateTime("2016-12-16T09:00:00"), DateTime("2017-01-16T09:00:00"), DateTime("2017-02-14T21:00:00")  …  DateTime("2098-12-16T09:00:

In [9]:
scopes_50_seasons = get_sliding_time_scopes_by_threshold(monthly_tl_data.time, 50)

37-element Vector{UnitRange{Int64}}:
 1:201
 4:205
 8:209
 12:213
 16:217
 20:221
 24:225
 28:229
 32:233
 36:237
 ⋮
 113:314
 117:318
 121:322
 125:327
 129:331
 133:335
 137:339
 141:343
 145:347

In [10]:
ivt_eof_data = calculate_eofs_of_tl_data(
    monthly_tl_data, 
    scopes_50_seasons, 
    2; 
    engine=:python, 
    reof=false, 
    center=true, 
    align_eof_with_mean=true, 
    align_pcs_with_mean=false, 
    weights=sqrt.(cos.(deg2rad.(monthly_tl_data.lats))), 
    eof_type=:normal,
    scale_with_eigenvals=true
)

Handled scope 1:201 out of 37 on thread 1
Handled scope 4:205 out of 37 on thread 1
Handled scope 8:209 out of 37 on thread 1
Handled scope 12:213 out of 37 on thread 1
Handled scope 16:217 out of 37 on thread 1
Handled scope 20:221 out of 37 on thread 1
Handled scope 24:225 out of 37 on thread 1
Handled scope 28:229 out of 37 on thread 1
Handled scope 32:233 out of 37 on thread 1
Handled scope 36:237 out of 37 on thread 1
Handled scope 40:241 out of 37 on thread 1
Handled scope 44:246 out of 37 on thread 1
Handled scope 48:250 out of 37 on thread 1
Handled scope 52:254 out of 37 on thread 1
Handled scope 56:258 out of 37 on thread 1
Handled scope 60:262 out of 37 on thread 1
Handled scope 64:266 out of 37 on thread 1
Handled scope 68:270 out of 37 on thread 1
Handled scope 72:274 out of 37 on thread 1
Handled scope 76:278 out of 37 on thread 1
Handled scope 80:282 out of 37 on thread 1
Handled scope 85:286 out of 37 on thread 1
Handled scope 89:290 out of 37 on thread 1
Handled scope 

Dict{String, Vector{EOFResult}} with 2 entries:
  "ssp126" => [EOFResult([-142.596 -60.2196 … 10.1789 7.75585; -137.487 -32.228…
  "ssp585" => [EOFResult([-71.9979 2.98634 … 12.9902 10.0804; -66.2574 42.7712 …

In [14]:
ps_eof_data = calculate_eofs_of_tl_data(
    ps_data, 
    scopes_50_seasons, 
    1; 
    engine=:python, 
    reof=false, 
    center=true, 
    align_eof_with_mean=true, 
    align_pcs_with_mean=false, 
    weights=sqrt.(cos.(deg2rad.(monthly_tl_data.lats))), 
    eof_type=:normal,
    scale_with_eigenvals=true
)

Handled scope 1:201 out of 37 on thread 1
Handled scope 4:205 out of 37 on thread 1
Handled scope 8:209 out of 37 on thread 1
Handled scope 12:213 out of 37 on thread 1
Handled scope 16:217 out of 37 on thread 1
Handled scope 20:221 out of 37 on thread 1
Handled scope 24:225 out of 37 on thread 1
Handled scope 28:229 out of 37 on thread 1
Handled scope 32:233 out of 37 on thread 1
Handled scope 36:237 out of 37 on thread 1
Handled scope 40:241 out of 37 on thread 1
Handled scope 44:246 out of 37 on thread 1
Handled scope 48:250 out of 37 on thread 1
Handled scope 52:254 out of 37 on thread 1
Handled scope 56:258 out of 37 on thread 1
Handled scope 60:262 out of 37 on thread 1
Handled scope 64:266 out of 37 on thread 1
Handled scope 68:270 out of 37 on thread 1
Handled scope 72:274 out of 37 on thread 1
Handled scope 76:278 out of 37 on thread 1
Handled scope 80:282 out of 37 on thread 1
Handled scope 85:286 out of 37 on thread 1
Handled scope 89:290 out of 37 on thread 1
Handled scope 

Dict{String, Vector{EOFResult}} with 2 entries:
  "ssp126" => [EOFResult([-1046.94 -1187.83 … 2886.88 2638.54; -947.161 -1109.4…
  "ssp585" => [EOFResult([863.202 984.147 … -2806.43 -2596.98; 789.093 927.23 ……

In [33]:


crosscorrelations = [findmax(abs.(crosscor(standardize(ZScoreTransform, ps_eof_data["ssp585"][i].temporal_modes[:, 1]), standardize(ZScoreTransform, ivt_eof_data["ssp585"][i].temporal_modes[:, 1]), -200:200))) for i in eachindex(scopes_50_seasons)]

37-element Vector{Tuple{Float64, Int64}}:
 (0.7625049973749771, 201)
 (0.7397573260413366, 201)
 (0.7490668647055344, 201)
 (0.7462365313429762, 201)
 (0.7581948159093151, 201)
 (0.7632417918685052, 201)
 (0.733875679598915, 201)
 (0.7072955887199428, 201)
 (0.6534639694063032, 201)
 (0.6526275421926124, 201)
 ⋮
 (0.5589771156467199, 201)
 (0.5504738106774838, 201)
 (0.551907629897672, 201)
 (0.6181822726481405, 201)
 (0.5940167439260987, 201)
 (0.6100935257165574, 201)
 (0.6215661498898126, 201)
 (0.6044656793903738, 201)
 (0.5824990570840811, 201)

In [37]:
crosscorrelations_126 = [findmax(abs.(crosscor(standardize(ZScoreTransform, ps_eof_data["ssp126"][i].temporal_modes[:, 1]), standardize(ZScoreTransform, ivt_eof_data["ssp126"][i].temporal_modes[:, 1]), -200:200))) for i in eachindex(scopes_50_seasons)]

37-element Vector{Tuple{Float64, Int64}}:
 (0.6622071772436333, 201)
 (0.6398173633695974, 201)
 (0.6353794287983177, 201)
 (0.6102344170763132, 201)
 (0.5782742727610636, 201)
 (0.6474737073070861, 201)
 (0.6088433504776555, 201)
 (0.614096747364633, 201)
 (0.6565152045162468, 201)
 (0.645864984838641, 201)
 ⋮
 (0.710289008785419, 201)
 (0.7198934285469002, 201)
 (0.7201176657215961, 201)
 (0.6849477319948856, 201)
 (0.6983063451172546, 201)
 (0.7006178614755, 201)
 (0.6982257577873604, 201)
 (0.7098426537522626, 201)
 (0.6903473510175815, 201)

In [34]:
Set([c[2] for c in crosscorrelations])

Set{Int64} with 1 element:
  201

In [29]:
corellations = [cor(standardize(ZScoreTransform, ps_eof_data["ssp585"][i].temporal_modes[:, 1]), standardize(ZScoreTransform, ivt_eof_data["ssp585"][i].temporal_modes[:, 1])) for i in eachindex(scopes_50_seasons)]

37-element Vector{Float64}:
  0.7625049973749771
 -0.7397573260413366
  0.7490668647055342
  0.7462365313429762
 -0.7581948159093153
  0.7632417918685054
  0.7338756795989151
  0.7072955887199428
 -0.6534639694063034
  0.6526275421926123
  ⋮
 -0.5589771156467199
  0.5504738106774838
  0.551907629897672
  0.6181822726481405
 -0.5940167439260987
 -0.6100935257165574
 -0.6215661498898126
 -0.6044656793903738
 -0.582499057084081

In [23]:
collect(-100:100)[101]

0

In [39]:
fig = Figure()

axis = Axis(fig[1, 1], title = "Crosscorrelation of IVT mode 1 and PS mode 1")

lines!(axis, 1:37, [c[1] for c in crosscorrelations], label = "ssp585")
lines!(axis, 1:37, [c[1] for c in crosscorrelations_126], label = "ssp126")
axislegend(axis)

fig

In [40]:
ps_data.time[end]

2101-01-01T00:00:00